In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, GridSearchCV
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_df = pd.read_csv('../csvs/train.csv')
test_df = pd.read_csv('../csvs/test.csv')

# Feature Engineering

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
# We can also describe categorical data
train_df.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,1601,G6,S
freq,1,577,7,4,644


In [7]:
train_df.groupby(train_df["Pclass"], as_index=False)["Survived"].mean()

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [8]:
train_df.groupby(train_df["Sex"], as_index=False)["Survived"].mean()

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [9]:
train_df.groupby(train_df["SibSp"], as_index=False)["Survived"].mean()

,SibSp,Survived
0,0,0.345395
1,1,0.535885
2,2,0.464286
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


In [10]:
train_df.groupby(train_df["Parch"], as_index=False)["Survived"].mean()

,Parch,Survived
0,0,0.343658
1,1,0.550847
2,2,0.500000
3,3,0.600000
4,4,0.000000
5,5,0.200000
6,6,0.000000


In [11]:
train_df["FamilySize"] = train_df["SibSp"] + train_df["Parch"] + 1
test_df["FamilySize"] = test_df["SibSp"] + test_df["Parch"] + 1
train_df.groupby(train_df["FamilySize"], as_index=False)["Survived"].mean()


,FamilySize,Survived
0,1,0.303538
1,2,0.552795
2,3,0.578431
3,4,0.724138
4,5,0.200000
5,6,0.136364
6,7,0.333333
7,8,0.000000
8,11,0.000000


In [12]:
family_map = {1: "Alone", 2: "Small", 3: "Small", 4: "Small", 5: "Medium", 6: "Medium", 7: "Large", 8: "Large", 9: "Large", 10: "Large"}
train_df["FamilyGroupSize"] = train_df["FamilySize"].map(family_map)
test_df["FamilyGroupSize"] = test_df["FamilySize"].map(family_map)
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,FamilyGroupSize
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,Small
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,Small
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,Alone
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,Small
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,Alone


In [13]:
train_df.groupby(train_df["Embarked"], as_index=False)["Survived"].mean()

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.336957


In [14]:
age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80]
age_cut = pd.cut(train_df["Age"], bins=age_bins)
train_df.groupby(age_cut, as_index=False)["Survived"].mean()

,Survived
0,0.593750
1,0.382609
2,0.365217
3,0.445161
4,0.383721
5,0.404762
6,0.235294
7,0.200000


In [15]:
fare_bins = [0, 100, 200, 300, 400, 500, 600]
fare_cut = pd.cut(train_df["Fare"], bins=fare_bins)
train_df.groupby(fare_cut, as_index=False)['Survived'].mean()

,Survived
0,0.366950
1,0.757576
2,0.647059
3,NaN
4,NaN
5,1.000000


In [16]:
train_df["Name"]

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object

In [17]:
train_df["Title"] = train_df["Name"].str.split(",", expand=True)[1].str.split(".", expand=True)[0].apply(lambda x : x.strip())
test_df["Title"] = test_df["Name"].str.split(",", expand=True)[1].str.split(".", expand=True)[0].apply(lambda x : x.strip())

In [18]:
train_df.groupby(train_df["Title"])["Survived"].agg(["count", "mean"])

,count,mean
Title,,
Capt,1,0.000000
Col,2,0.500000
Don,1,0.000000
Dr,7,0.428571
Jonkheer,1,0.000000
Lady,1,1.000000
Major,2,0.500000
Master,40,0.575000
Miss,182,0.697802


- military - Capt, Col, Major
- noble - Jonkheer, the Countess, Don, Lady, Sir
- unmarried Female - Mlle, Ms, Mme

In [19]:
title_category = {"Capt": "military", "Col": "military", "Major": "military", "Jonkheer": "noble", "the Countess": "noble", "Don": "noble", "Lady": "noble", "Sir": "noble", "Mlle": "unmarried Female", "Ms": "unmarried Female", "Mme": "unmarried Female"}
train_df["Title"] = train_df["Title"].replace(title_category)
test_df["Title"] = test_df["Title"].replace(title_category)

In [20]:
train_df.groupby(train_df["Title"])["Survived"].agg(["count", 'mean'])

,count,mean
Title,,
Dr,7,0.428571
Master,40,0.575000
Miss,182,0.697802
Mr,517,0.156673
Mrs,125,0.792000
Rev,6,0.000000
military,5,0.400000
noble,5,0.600000
unmarried Female,4,1.000000


In [21]:
train_df["Ticket"]

0             A/5 21171
1              PC 17599
2      STON/O2. 3101282
3                113803
4                373450
             ...       
886              211536
887              112053
888          W./C. 6607
889              111369
890              370376
Name: Ticket, Length: 891, dtype: object

In [22]:
train_df["TicketNumber"] = train_df["Ticket"].apply(lambda x: x.split()[-1])
test_df["TicketNumber"] = test_df["Ticket"].apply(lambda x: x.split()[-1])
train_df['TicketNumber']

0        21171
1        17599
2      3101282
3       113803
4       373450
        ...   
886     211536
887     112053
888       6607
889     111369
890     370376
Name: TicketNumber, Length: 891, dtype: object

In [23]:
train_df.groupby(["TicketNumber"], as_index=False)["Survived"].agg(['count', 'mean']).sort_values('count', ascending=False)


,TicketNumber,count,mean
196,2343,7,0.000000
94,1601,7,0.714286
464,347082,7,0.000000
468,347088,6,0.000000
358,3101295,6,0.000000
...,...,...,...
55,1166,1,0.000000
311,28424,1,0.000000
312,28425,1,0.000000
313,28551,1,1.000000


In [24]:
train_df["TicketNumberCounts"] = train_df.groupby(['TicketNumber'])["TicketNumber"].transform('count')
test_df["TicketNumberCounts"] = test_df.groupby(['TicketNumber'])["TicketNumber"].transform('count')

In [25]:
train_df.groupby(["TicketNumberCounts"], as_index=True)['Survived'].agg(['count', 'mean']).sort_values('count', ascending=False)

,count,mean
TicketNumberCounts,,
1,544,0.295956
2,188,0.569149
3,66,0.712121
4,44,0.500000
7,21,0.238095
6,18,0.000000
5,10,0.000000


In [26]:
train_df["Ticket"]

0             A/5 21171
1              PC 17599
2      STON/O2. 3101282
3                113803
4                373450
             ...       
886              211536
887              112053
888          W./C. 6607
889              111369
890              370376
Name: Ticket, Length: 891, dtype: object

In [27]:
train_df["TicketPrefix"] = np.where(train_df["Ticket"].str.split(' ', expand=True, n=1)[1].notna(), train_df["Ticket"].str.split(' ', expand=True, n=1)[0].apply(lambda x: x.strip()), "Blank")
test_df["TicketPrefix"] = np.where(test_df["Ticket"].str.split(' ', expand=True, n=1)[1].notna(), test_df["Ticket"].str.split(' ', expand=True, n=1)[0].apply(lambda x: x.strip()), "Blank")

In [28]:
train_df["TicketPrefix"].value_counts()

TicketPrefix
Blank         665
PC             60
C.A.           27
STON/O         12
A/5            10
W./C.           9
CA.             8
SOTON/O.Q.      8
A/5.            7
SOTON/OQ        7
STON/O2.        6
CA              6
C               5
S.O.C.          5
SC/PARIS        5
F.C.C.          5
SC/Paris        4
A/4.            3
PP              3
A/4             3
S.O./P.P.       3
SC/AH           3
A./5.           2
P/PP            2
A.5.            2
WE/P            2
SOTON/O2        2
S.C./PARIS      2
S.C./A.4.       1
Fa              1
S.O.P.          1
SO/C            1
S.P.            1
A4.             1
W.E.P.          1
A/S             1
SC              1
SW/PP           1
SCO/W           1
W/C             1
S.W./PP         1
F.C.            1
C.A./SOTON      1
Name: count, dtype: int64

In [29]:
train_df["TicketLocation"] = train_df["TicketPrefix"].replace({
    'SOTON/O.Q': 'SOTON/OQ',
    'C.A': 'CA',
    'CA.': 'CA',
    'S.C./PARIS': 'SC/Paris',
    'S.C./PARIS': 'SC/Paris',
    'A/4': 'A/4',
    'A/5': 'A/5',
    'A/5.': 'A/5',
    'A./5': 'A/5',
    'W./C.': 'W/C'
})
test_df["TicketLocation"] = test_df["TicketPrefix"].replace({
    'SOTON/O.Q': 'SOTON/OQ',
    'C.A': 'CA',
    'CA.': 'CA',
    'S.C./PARIS': 'SC/Paris',
    'S.C./PARIS': 'SC/Paris',
    'A/4': 'A/4',
    'A/5': 'A/5',
    'A/5.': 'A/5',
    'A./5': 'A/5',
    'W./C.': 'W/C'
})

In [30]:
train_df.groupby(train_df["TicketLocation"])["Survived"].agg(['count', 'mean']).count()

count    39
mean     39
dtype: int64

In [31]:
train_df["Cabin"]

0       NaN
1       C85
2       NaN
3      C123
4       NaN
       ... 
886     NaN
887     B42
888     NaN
889    C148
890     NaN
Name: Cabin, Length: 891, dtype: object

In [32]:
train_df["Cabin"].isnull().sum()

np.int64(687)

In [33]:
train_df["Cabin"].fillna("U", inplace=True)
test_df["Cabin"].fillna("U", inplace=True)

In [34]:
split_cabin = train_df["Cabin"].str.split(' ', expand=True)[0]
split_cabin_test = test_df["Cabin"].str.split(' ', expand=True)[0]
split_cabin.str[0].value_counts()


0
U    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: count, dtype: int64

In [35]:
train_df["Cabin"] = split_cabin.str[0]
test_df["Cabin"] = split_cabin_test.str[0]
train_df["Cabin"]


0      U
1      C
2      U
3      C
4      U
      ..
886    U
887    B
888    U
889    C
890    U
Name: Cabin, Length: 891, dtype: object

In [36]:
train_df["Cabin"].value_counts()

Cabin
U    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: count, dtype: int64

In [37]:
train_df.groupby(train_df["Cabin"], as_index=True)["Survived"].agg(['count', 'mean'])

,count,mean
Cabin,,
A,15,0.466667
B,47,0.744681
C,59,0.593220
D,33,0.757576
E,32,0.750000
F,13,0.615385
G,4,0.500000
T,1,0.000000
U,687,0.299854


In [38]:
train_df["Cabin_Assigned"] = train_df["Cabin"].apply(lambda x: 0 if x in ['U'] else 1)
test_df["Cabin_Assigned"] = train_df["Cabin"].apply(lambda x: 0 if x in ['U'] else 1)
train_df["Cabin_Assigned"]

0      0
1      1
2      0
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Cabin_Assigned, Length: 891, dtype: int64

In [39]:
train_df.groupby(train_df["Cabin_Assigned"], as_index=True)["Survived"].agg(['count', 'mean'])

,count,mean
Cabin_Assigned,,
0,687,0.299854
1,204,0.666667


In [40]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,FamilyGroupSize,Title,TicketNumber,TicketNumberCounts,TicketPrefix,TicketLocation,Cabin_Assigned
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U,S,2,Small,Mr,21171,1,A/5,A/5,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C,2,Small,Mrs,17599,1,PC,PC,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,U,S,1,Alone,Miss,3101282,1,STON/O2.,STON/O2.,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C,S,2,Small,Mrs,113803,2,Blank,Blank,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,U,S,1,Alone,Mr,373450,1,Blank,Blank,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,U,S,1,Alone,Rev,211536,1,Blank,Blank,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B,S,1,Alone,Miss,112053,1,Blank,Blank,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,U,S,4,Small,Miss,6607,2,W./C.,W/C,0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,C,1,Alone,Mr,111369,1,Blank,Blank,1


In [41]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PassengerId         891 non-null    int64  
 1   Survived            891 non-null    int64  
 2   Pclass              891 non-null    int64  
 3   Name                891 non-null    object 
 4   Sex                 891 non-null    object 
 5   Age                 714 non-null    float64
 6   SibSp               891 non-null    int64  
 7   Parch               891 non-null    int64  
 8   Ticket              891 non-null    object 
 9   Fare                891 non-null    float64
 10  Cabin               891 non-null    object 
 11  Embarked            889 non-null    object 
 12  FamilySize          891 non-null    int64  
 13  FamilyGroupSize     884 non-null    object 
 14  Title               891 non-null    object 
 15  TicketNumber        891 non-null    object 
 16  TicketNu

In [42]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PassengerId         418 non-null    int64  
 1   Pclass              418 non-null    int64  
 2   Name                418 non-null    object 
 3   Sex                 418 non-null    object 
 4   Age                 332 non-null    float64
 5   SibSp               418 non-null    int64  
 6   Parch               418 non-null    int64  
 7   Ticket              418 non-null    object 
 8   Fare                417 non-null    float64
 9   Cabin               418 non-null    object 
 10  Embarked            418 non-null    object 
 11  FamilySize          418 non-null    int64  
 12  FamilyGroupSize     414 non-null    object 
 13  Title               418 non-null    object 
 14  TicketNumber        418 non-null    object 
 15  TicketNumberCounts  418 non-null    int64  
 16  TicketPr

In [43]:
train_df["Age"].fillna(train_df["Age"].mean(), inplace=True)
test_df["Age"].fillna(test_df["Age"].mean(), inplace=True)
test_df["Fare"].fillna(test_df["Fare"].mean(), inplace=True)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PassengerId         891 non-null    int64  
 1   Survived            891 non-null    int64  
 2   Pclass              891 non-null    int64  
 3   Name                891 non-null    object 
 4   Sex                 891 non-null    object 
 5   Age                 891 non-null    float64
 6   SibSp               891 non-null    int64  
 7   Parch               891 non-null    int64  
 8   Ticket              891 non-null    object 
 9   Fare                891 non-null    float64
 10  Cabin               891 non-null    object 
 11  Embarked            889 non-null    object 
 12  FamilySize          891 non-null    int64  
 13  FamilyGroupSize     884 non-null    object 
 14  Title               891 non-null    object 
 15  TicketNumber        891 non-null    object 
 16  TicketNu

# Model Pipeline

In [44]:
ohe = OneHotEncoder(sparse_output=False)
ode = OrdinalEncoder()
SI = SimpleImputer(strategy='most_frequent') # Embarked


In [45]:
ode_cols = ["FamilyGroupSize"]
ohe_cols = ["Sex", "Embarked"]

In [46]:
X = train_df.drop(["Survived"], axis=1)
y = train_df["Survived"]
X_test = test_df

In [47]:
# Split the training set into train, validate and test
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [48]:
X_train.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,FamilySize,TicketNumberCounts,Cabin_Assigned
count,668.00000,668.000000,668.000000,668.000000,668.000000,668.000000,668.00000,668.000000,668.000000
mean,444.61976,2.312874,29.886234,0.464072,0.375749,31.177469,1.83982,1.764970,0.218563
std,259.24582,0.831906,12.959192,0.999353,0.832877,47.457877,1.51942,1.325621,0.413581
min,1.00000,1.000000,0.420000,0.000000,0.000000,0.000000,1.00000,1.000000,0.000000
25%,221.75000,2.000000,22.000000,0.000000,0.000000,7.895800,1.00000,1.000000,0.000000
50%,440.50000,3.000000,29.699118,0.000000,0.000000,13.860400,1.00000,1.000000,0.000000
75%,669.25000,3.000000,36.000000,1.000000,0.000000,30.500000,2.00000,2.000000,0.000000
max,891.00000,3.000000,80.000000,8.000000,6.000000,512.329200,11.00000,7.000000,1.000000


In [49]:

#* Apply a list of transformers to ordinal data
ordinal_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    # ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=1)) - Throws error because 1 is already used as an encoded value for the known types
    ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan, dtype=float))
])



In [50]:

#* Apply a list of transformers to Nominal data
ohe_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

In [51]:
# * Appy transformers to Specific Numerical columns
col_trans = ColumnTransformer(
    transformers=[
        ("impute", SimpleImputer(strategy="mean"), ["Age"]),
        ("ordinal_pipeline", ordinal_pipeline, ode_cols),
        ("ohe_pipeline", ohe_pipeline, ohe_cols),
        (
            "passthrough",
            "passthrough",
            [
                "Pclass",
                "TicketNumberCounts",
                "Age",
                "Cabin_Assigned",
                "Fare",
                "FamilySize",
            ],
        ),
    ],
    remainder="drop",
    n_jobs=-1,
)

# Models

## Random Forests

In [52]:
rfc = RandomForestClassifier()

In [53]:
param_grid = {
    'n_estimators': [100, 150, 200],
    'min_samples_split': [5, 10, 15],
    'max_depth': [8, 9, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy', 'log_loss']
}

In [54]:
# cv = StratifiedKFold(random_state=42, shuffle=True)
cv = StratifiedKFold(n_splits=5)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=cv)

In [55]:
pipefinalrfc = make_pipeline(col_trans, CV_rfc)
pipefinalrfc.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('gridsearchcv', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('impute', ...), ('ordinal_pipeline', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output

In [56]:
y_pred = pipefinalrfc.predict(X_valid)


In [57]:
print(f"Best Params: {CV_rfc.best_params_}")
print(f"Best Score: {CV_rfc.best_score_}")
print(f"Best Estimator: {CV_rfc.best_estimator_}")

Best Params: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Best Score: 0.8398047357198968
Best Estimator: RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=2,
                       min_samples_split=5)


In [58]:
print(classification_report(y_valid, y_pred))
print(confusion_matrix(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.90      0.85       137
           1       0.80      0.64      0.71        86

    accuracy                           0.80       223
   macro avg       0.80      0.77      0.78       223
weighted avg       0.80      0.80      0.79       223

[[123  14]
 [ 31  55]]


## Decision Tree

In [59]:
dtc = DecisionTreeClassifier()

In [60]:
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'min_samples_split': [2, 5, 10, 15],
    'max_depth': [8, 9, 10, 15, 20],
    'min_samples_leaf': [1,2 ,4]
}

In [61]:
cv = StratifiedKFold(n_splits=5)
CV_dtc = GridSearchCV(estimator=dtc, param_grid=param_grid, cv=cv)

In [62]:
pipefinaldtc = make_pipeline(col_trans, CV_dtc)
pipefinaldtc.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('gridsearchcv', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('impute', ...), ('ordinal_pipeline', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output

In [63]:
print(f"Best Params: {CV_dtc.best_params_}")
print(f"Best Score: {CV_dtc.best_score_}")
print(f"Best Estimator: {CV_dtc.best_estimator_}")

Best Params: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5}
Best Score: 0.8218381775333856
Best Estimator: DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_split=5)


## KNN

In [64]:
knn = KNeighborsClassifier()

In [65]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'p': [1, 2, 3, 5],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

In [66]:
cv = StratifiedKFold(n_splits=5)
CV_knn = GridSearchCV(estimator=knn, param_grid=param_grid, cv=cv)

In [67]:
pipefinalknn = make_pipeline(col_trans, CV_knn)
pipefinalknn.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('gridsearchcv', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('impute', ...), ('ordinal_pipeline', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output

In [68]:
print(f"Best Params: {CV_knn.best_params_}")
print(f"Best Score: {CV_knn.best_score_}")
print(f"Best Estimator: {CV_knn.best_estimator_}")

Best Params: {'algorithm': 'auto', 'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
Best Score: 0.7860060599259342
Best Estimator: KNeighborsClassifier(n_neighbors=9, p=1, weights='distance')


## SVC

In [3]:
svc = SVC()

NameError: name 'SVC' is not defined

In [1]:
param_grid = {
    'C': [100, 10, 1.0,0.1, 0.01, 0.001],
    'kernel': ['linear' 'rbf'], # Avoid pre-computed, which expects a square kernel matrix - Also avoid poly and sigmoid(make the model expensive)
    'gamma': ['scale', 'auto']
}

In [2]:
cv = StratifiedKFold(n_splits=5)
CV_svc = GridSearchCV(estimator=svc, param_grid=param_grid, cv=cv)

NameError: name 'StratifiedKFold' is not defined

In [ ]:
pipefinalsvc = make_pipeline(col_trans, CV_svc)
pipefinalsvc.fit(X_train, y_train)

In [73]:
print(f"Best Params: {CV_svc.best_params_}")
print(f"Best Score: {CV_svc.best_score_}")
print(f"Best Estimator: {CV_svc.best_estimator_}")

Best Params: {'C': 100, 'gamma': 'scale'}
Best Score: 0.8144540455616653
Best Estimator: SVC(C=100)


## Logistic Regression

In [74]:
lr = LogisticRegression()

In [75]:
param_grid = {
    'C': [100, 1.0, 0.1, 0.01, 0.001]
}

In [76]:
cv = StratifiedKFold(n_splits=5)
CV_lr = GridSearchCV(estimator=lr, param_grid=param_grid, cv=cv)

In [77]:
pipefinallr = make_pipeline(col_trans, CV_lr)
pipefinallr.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('gridsearchcv', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('impute', ...), ('ordinal_pipeline', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output

In [78]:
print(f"Best Params: {CV_lr.best_params_}")
print(f"Best Score: {CV_lr.best_score_}")
print(f"Best Estimator: {CV_lr.best_estimator_}")

Best Params: {'C': 1.0}
Best Score: 0.8129053978229155
Best Estimator: LogisticRegression()


## Naive Bayes

In [79]:
gnb = GaussianNB()

In [80]:
param_grid = {
    'var_smoothing': [0.0000001, 0.000000001, 0.0000000001]
}

In [81]:
cv = StratifiedKFold(n_splits=5)
CV_gnb = GridSearchCV(estimator=gnb, param_grid=param_grid, cv=cv)

In [82]:
pipefinalgnb = make_pipeline(col_trans, CV_gnb)
pipefinalgnb.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('gridsearchcv', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('impute', ...), ('ordinal_pipeline', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output

In [83]:
print(f"Best Params: {CV_gnb.best_params_}")
print(f"Best Score: {CV_gnb.best_score_}")
print(f"Best Estimator: {CV_gnb.best_estimator_}")

Best Params: {'var_smoothing': 1e-07}
Best Score: 0.7964313769498372
Best Estimator: GaussianNB(var_smoothing=1e-07)


In [84]:
y_pred1 = pipefinalrfc.predict(X_test)
y_pred2 = pipefinaldtc.predict(X_test)
y_pred3 = pipefinalknn.predict(X_test)
y_pred4 = pipefinalsvc.predict(X_test)
y_pred5 = pipefinallr.predict(X_test)
y_pred6 = pipefinalgnb.predict(X_test)

## Submission

In [85]:
submission1 = pd.DataFrame({
    'PassengerId': test_df["PassengerId"],
    "Survived": y_pred1
})
submission2 = pd.DataFrame({
    'PassengerId': test_df["PassengerId"],
    "Survived": y_pred2
})
submission3 = pd.DataFrame({
    'PassengerId': test_df["PassengerId"],
    "Survived": y_pred3
})
submission4 = pd.DataFrame({
    'PassengerId': test_df["PassengerId"],
    "Survived": y_pred4
})
submission5 = pd.DataFrame({
    'PassengerId': test_df["PassengerId"],
    "Survived": y_pred5
})
submission6 = pd.DataFrame({
    'PassengerId': test_df["PassengerId"],
    "Survived": y_pred6
})

In [88]:
# submission1.to_csv('/kaggle/working/submission1102_1.csv', index=False)
# submission2.to_csv('/kaggle/working/submission1102_2.csv', index=False)
# submission3.to_csv('/kaggle/working/submission1102_3.csv', index=False)
# submission4.to_csv('/kaggle/working/submission1102_4.csv', index=False)
# submission5.to_csv('/kaggle/working/submission1102_5.csv', index=False)
# submission6.to_csv('/kaggle/working/submission1102_5.csv', index=False)

submission1.to_csv('./', index=False)
submission2.to_csv('./', index=False)
submission3.to_csv('./', index=False)
submission4.to_csv('./', index=False)
submission5.to_csv('./', index=False)
submission6.to_csv('./', index=False)


IsADirectoryError: [Errno 21] Is a directory: './'